In [11]:
#import tensorflow as tf
#from keras import layers
#from keras import preprocessing
#import tensorFlowTutorial as t
#from tensorboard.plugins import projector
import dataProsessing as dp
import numpy as np
import os
import io
import json
import math, random, time
from numpy import linalg as LA
import music21  
np.set_printoptions(suppress=True)
from tabulate import tabulate
import collections as coll
#import inflect

In [2]:
home_dir = os.environ['HOME']
path_dir: str = home_dir + r"/MAT180/MAT_180_ML_Projects/corpus-of-chords"
os.chdir(path_dir)

In [3]:
def k_means(M,k,tol = .01, max_iters = 100):
    ################################## your code goes here ##########################################

    #inits
    steps = 0
    prev = tol+1
    current = 0
    m = len(M)
    n = len(M[0,:])
    initialize = random.sample(range(n),k)
    KM = M[:,initialize].astype(np.float32) 
    partition = [0] * n
    
    while (abs(prev-current) > tol) and (steps < max_iters):
        steps += 1
        for i in range(n):
            # find closest column of KM
            prev_diff = -1
            for j in range(k):
                diff = LA.norm(M[:,i]-KM[:,j],2)
                if prev_diff <0 or diff<prev_diff:
                    prev_diff = diff
                    partition[i] = j
        
        for j in range(k):
            # KM[:,j] = take mean of cols such that partition[s] == j
            s_indices = [ s for s in range(n) if partition[s] == j]
            S = M[:,s_indices]
            KM[:,j] = S.sum(axis = 1) / len(s_indices)
        
        prev = current
        current = sum([LA.norm(M[:,i]-KM[:,partition[i]]) for i in range(n)])
        
        print(f'after {steps} steps the residue is {current}')
        
    return(KM, partition)


Load corpi of chords:
- vocab: dictionary with key:value as (chord vector in midi format):index
- inv_vocab: chords as vector using midi numbers
- weights: weight vectors outputted from word2vec, ordered corresponding to inv_vocab

In [4]:
#set model_name manually
model_name = "d_128 n_15"

datasetName = "Jsb16thSeparated(t_60_rr_re)"

#add option for / vs \ check TODO
vocab = dataset = json.load(open("Data/Corpi/" + datasetName + "/" + datasetName +"_vocab.json"))
inv_vocab = vocab["inv_vocab"]
vocab = vocab["vocab"]
weights = json.load(open("Data/Corpi/" + datasetName + "/weights/" + model_name))
weights_km = np.array(weights).T

Gather chord labels and format ...

- weights: np array with weights as np array listed by inv_vocab index
- labels_km: dictionary with chord:label

In [5]:
labels_km = dp.getChordLabels(inv_vocab,True,True) #gets (chord name, chord root, chord quality)

0th step
300th step
600th step
900th step
1200th step
1500th step
1800th step
2100th step
2400th step
2700th step
3000th step
3300th step
3600th step
3900th step
4200th step
4500th step
4800th step
5100th step
5400th step
5700th step
6000th step
6300th step
6600th step
6900th step


In [6]:
# input needed (choose the most appropriate value for k)
# may have to run this a few times until it gets a good enough initialization

k = 24
KM,partition = k_means(weights_km,k,.0001,100)

after 1 steps the residue is 14610.99219383493
after 2 steps the residue is 14227.813035734067
after 3 steps the residue is 14069.09418348907
after 4 steps the residue is 13988.83562713949
after 5 steps the residue is 13948.696061098548
after 6 steps the residue is 13906.227350064597
after 7 steps the residue is 13888.0882680225
after 8 steps the residue is 13875.540147936248
after 9 steps the residue is 13864.859167671468
after 10 steps the residue is 13855.903847232805
after 11 steps the residue is 13848.579103021193
after 12 steps the residue is 13844.134047593003
after 13 steps the residue is 13841.210326606559
after 14 steps the residue is 13839.47262341342
after 15 steps the residue is 13838.49124642667
after 16 steps the residue is 13837.800073966664
after 17 steps the residue is 13837.33000981476
after 18 steps the residue is 13836.906161895675
after 19 steps the residue is 13836.695198637992
after 20 steps the residue is 13836.444536962683
after 21 steps the residue is 13836.2

In [144]:
def getFreqs(indices, data):
    datacount = coll.Counter([data[i] for i in indices])
    total = sum(datacount.values())
    ## TODO check duplicates here?
    freq = [[data[i],round(datacount[data[i]]/total,4)] for i in indices]
    return freq

In [134]:
def mostcommon(n,freq):
    order = np.argsort(np.array(freq)[:,1])
    this = [freq[o] for o in order[len(order)-n:len(order)]]
    return this

In [135]:
def tabulate_partitions(feature,freqs,n,k):
    title = f'Partitions, {feature}'
    tabledata = []
    for p in range(k):
        tabledata += [[p] + [partsizes[p]] + mostcommon(n,freqs[p])]
    headers = ['Partition','part. size']
    for i in range(n): #TODO: format
        headers += [f'{i} most common']
    return (title,tabledata,headers)

In [145]:
#sets per partition information

namefreqs = []
rootfreqs = []
qualfreqs = []
partsizes = []
for p in range(k):
    p_indices = [i for i in range(len(partition)) if partition[i]==p]
    namefreqs += [getFreqs(p_indices,labels_km[:,0])]
    rootfreqs += [getFreqs(p_indices,labels_km[:,1])]
    qualfreqs += [getFreqs(p_indices,labels_km[:,2])]
    partsizes += [round(len(p_indices)/len(weights),4)]



In [148]:
title,tabledata,headers = tabulate_partitions("ChordName",rootfreqs,4,k) #buggie doesn't handle well when n > #eles in partition
print(title)
print(tabulate(tabledata,headers))

Partitions, ChordName
  Partition    part. size  0 most common    1 most common    2 most common    3 most common
-----------  ------------  ---------------  ---------------  ---------------  ---------------
          0        0.0006  ['F', 0.25]      ['E', 0.25]      ['D', 0.25]      ['B-', 0.25]
          1        0.0407  ['E-', 0.2439]   ['E-', 0.2439]   ['E-', 0.2439]   ['E-', 0.2439]
          2        0.0378  ['G', 0.188]     ['G', 0.188]     ['G', 0.188]     ['G', 0.188]
          3        0.0836  ['F', 0.1409]    ['F', 0.1409]    ['F', 0.1409]    ['F', 0.1409]
          4        0.0319  ['F', 0.1733]    ['F', 0.1733]    ['F', 0.1733]    ['F', 0.1733]
          5        0.1154  ['C', 0.1378]    ['C', 0.1378]    ['C', 0.1378]    ['C', 0.1378]
          6        0.0256  ['B-', 0.2167]   ['B-', 0.2167]   ['B-', 0.2167]   ['B-', 0.2167]
          7        0.0302  ['C', 0.2535]    ['C', 0.2535]    ['C', 0.2535]    ['C', 0.2535]
          8        0.0434  ['A', 0.1797]    ['A', 0.1797